## Data Import and Preprocessing

In [1]:

import pandas as pd
from matplotlib import pyplot as plt

# Define Excel File Constant, and import function

EXCEL_FILE = "data/Muesli Project raw data 21-3.xlsx"

def import_xls(file, sheet, header=0):
    df = pd.read_excel(file, sheet_name = sheet, header=header)
    return df

    

In [2]:
# Import Excel Sheets as separate DFs

df_orders = import_xls(EXCEL_FILE, "Orders", header=1)
df_order_process = import_xls(EXCEL_FILE, "Order Process Data")
df_intern = import_xls(EXCEL_FILE, "InternData Study")
df_campaign = import_xls(EXCEL_FILE, "Campaign Data")
df_list = [df_orders, df_order_process, df_intern, df_campaign]


In [3]:
# Define Column Renaming Function
def column_rename(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [4]:
# Rename all DF columns
for df in df_list:
    column_rename(df)
#display(df_orders.head())


In [5]:
# Check for duplicates
for idx, df in enumerate(df_list):
    print(idx)
    print(f" Total order_id count is {df['order_id'].count()}")
    print(f" Total unique order_id count is {df['order_id'].nunique()}")
# 0 df_orders, 1 df_order_process, 2 df_intern, 3 df_campaign

0
 Total order_id count is 9994
 Total unique order_id count is 5009
1
 Total order_id count is 3003
 Total unique order_id count is 3002
2
 Total order_id count is 290
 Total unique order_id count is 204
3
 Total order_id count is 333
 Total unique order_id count is 333


In [6]:
# Define duplicate row removal on order_id function  
def duplicate_id_removal(df):
    df.drop_duplicates(subset=['order_id'], inplace=True)
    df.reset_index

In [7]:
# Remove duplicates for all DFs
for df in df_list:
     duplicate_id_removal(df)

In [8]:
# See if assumption about all order IDs being included in the order process tracking is correct
num_unique_dif = df_orders['order_id'].count() - df_order_process['order_id'].count()
print(f"We don't have all the order ids in the order process dataset. \nContrary to company assumptions there is a difference of {num_unique_dif} untracked orders")

# Investigate if all orders from 2/1/2019 on are tracked then

df_orders_recent = df_orders[df_orders['order_date'] >= df_order_process["order_date"][0]]
display(df_orders_recent.info())
display(df_order_process.info())


# df_orders['order_id'].isin(df_order_process['order_id']).count()
# ~ = not in

We don't have all the order ids in the order process dataset. 
Contrary to company assumptions there is a difference of 2007 untracked orders
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 4095 to 9993
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        3002 non-null   object        
 1   order_date      3002 non-null   datetime64[ns]
 2   customer_id     3002 non-null   object        
 3   customer_name   3002 non-null   object        
 4   origin_channel  3002 non-null   object        
 5   country/region  3002 non-null   object        
 6   city            3002 non-null   object        
 7   state           3002 non-null   object        
 8   postal_code     2998 non-null   float64       
 9   region          3002 non-null   object        
 10  category        3002 non-null   object        
 11  sub-category    3002 non-null   object        
 12  product_id     

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 0 to 3002
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            3002 non-null   object        
 1   order_date          3002 non-null   datetime64[ns]
 2   on_truck_scan_date  3002 non-null   datetime64[ns]
 3   processing_mode     3002 non-null   object        
dtypes: datetime64[ns](2), object(2)
memory usage: 181.8+ KB


None

In [9]:
df_list = [df_orders, df_order_process, df_intern, df_campaign]

df_all = pd.merge(df_orders, df_order_process, how='left', on='order_id')
df_all = pd.merge(df_all, df_intern, how='left', on='order_id')
df_all = pd.merge(df_all, df_campaign, how='left', on='order_id')

df = pd.merge(df_orders, df_order_process, how='inner', on='order_id') 

In [10]:
# Doublecheck if we have the same amount of unique order ids in merged df as in the df_order_process
display(df['order_id'].nunique())

display(df_all.head())
display(df.head())

display(df_all.info())
display(df.info()) 

3002

,order_id,order_date_x,customer_id,customer_name_x,origin_channel,country/region,city,state,postal_code,region,...,sub-category,product_id,quantity,order_date_y,on_truck_scan_date,processing_mode,ready_to_ship_date,pickup_date,arrival_scan_date,customer_name_y
0,CA-2017-103800,2017-01-03,DP-13000,Darren Powers,Email,United States,Houston,Texas,77095.0,Central,...,Nuts and more,OFF-PA-10000174,2.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
1,CA-2017-112326,2017-01-04,PO-19195,Phillina Ober,Facebook,United States,Naperville,Illinois,60540.0,Central,...,No Taste All Power,OFF-LA-10003223,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
2,CA-2017-141817,2017-01-05,MB-18085,Mick Brown,Email,United States,Philadelphia,Pennsylvania,19143.0,East,...,Super Mega Protein,OFF-AR-10003478,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
3,CA-2017-106054,2017-01-06,JO-15145,Jack O'Briant,Sales,United States,Athens,Georgia,30605.0,South,...,Super Mega Protein,OFF-AR-10002399,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN
4,CA-2017-130813,2017-01-06,LS-17230,Lycoris Saunders,Email,United States,Los Angeles,California,90049.0,West,...,Nuts and more,OFF-PA-10002005,3.0,NaT,NaT,NaN,NaT,NaT,NaT,NaN


,order_id,order_date_x,customer_id,customer_name,origin_channel,country/region,city,state,postal_code,region,category,sub-category,product_id,quantity,order_date_y,on_truck_scan_date,processing_mode
0,CA-2019-160304,2019-01-02,BM-11575,Brendan Murry,Sales,United States,Gaithersburg,Maryland,20877.0,East,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-02,2019-01-09,Standard Processing
1,CA-2019-105207,2019-01-03,BO-11350,Bill Overfelt,Sales,United States,Broken Arrow,Oklahoma,74012.0,Central,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-03,2019-01-09,Standard Processing
2,CA-2019-125206,2019-01-03,LR-16915,Lena Radford,Email,United States,Los Angeles,California,90045.0,West,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-03,2019-01-07,Express
3,US-2019-116365,2019-01-03,CA-12310,Christine Abelman,Sales,United States,San Antonio,Texas,78207.0,Central,Special Projects Muesil,Gluten Free,TEC-AC-10002217,2.0,2019-01-03,2019-01-09,Standard Processing
4,CA-2019-158211,2019-01-04,BP-11185,Ben Peterman,Sales,United States,Philadelphia,Pennsylvania,19143.0,East,Power Muesli,Super Fibre Boost,OFF-BI-10002026,6.0,2019-01-04,2019-01-09,Standard Processing


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5009 entries, 0 to 5008
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            5009 non-null   object        
 1   order_date_x        5009 non-null   datetime64[ns]
 2   customer_id         5009 non-null   object        
 3   customer_name_x     5009 non-null   object        
 4   origin_channel      5009 non-null   object        
 5   country/region      5009 non-null   object        
 6   city                5009 non-null   object        
 7   state               5009 non-null   object        
 8   postal_code         5003 non-null   float64       
 9   region              5009 non-null   object        
 10  category            5009 non-null   object        
 11  sub-category        5009 non-null   object        
 12  product_id          5009 non-null   object        
 13  quantity            5009 non-null   float64     

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 0 to 3001
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            3002 non-null   object        
 1   order_date_x        3002 non-null   datetime64[ns]
 2   customer_id         3002 non-null   object        
 3   customer_name       3002 non-null   object        
 4   origin_channel      3002 non-null   object        
 5   country/region      3002 non-null   object        
 6   city                3002 non-null   object        
 7   state               3002 non-null   object        
 8   postal_code         2998 non-null   float64       
 9   region              3002 non-null   object        
 10  category            3002 non-null   object        
 11  sub-category        3002 non-null   object        
 12  product_id          3002 non-null   object        
 13  quantity            3002 non-null   float64     

None

In [11]:
df["order_id"].nunique()
drop_list = ['customer_name', 'country/region', 'city', 'state', 'postal_code', 'region'] 
df = df.drop(drop_list, axis=1)


In [12]:
display(df.head(10))


,order_id,order_date_x,customer_id,origin_channel,category,sub-category,product_id,quantity,order_date_y,on_truck_scan_date,processing_mode
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-02,2019-01-09,Standard Processing
1,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-03,2019-01-09,Standard Processing
2,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-03,2019-01-07,Express
3,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10002217,2.0,2019-01-03,2019-01-09,Standard Processing
4,CA-2019-158211,2019-01-04,BP-11185,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10002026,6.0,2019-01-04,2019-01-09,Standard Processing
5,US-2019-164630,2019-01-04,EB-13975,Sales,Special Projects Muesil,Organic,TEC-CO-10000971,4.0,2019-01-04,2019-01-11,Standard Processing
6,CA-2019-134474,2019-01-05,AJ-10795,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10001714,6.0,2019-01-05,2019-01-11,Standard Processing
7,CA-2019-101938,2019-01-07,DW-13480,Facebook,Power Muesli,Super Mega Protein,OFF-AR-10003696,1.0,2019-01-07,2019-01-16,Standard Processing
8,CA-2019-158806,2019-01-07,NM-18520,Email,Power Muesli,Nuts and more,OFF-PA-10004621,5.0,2019-01-07,2019-01-14,Standard Processing
9,US-2019-100461,2019-01-08,JO-15145,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10001460,7.0,2019-01-08,2019-01-16,Standard Processing


In [13]:
df[df["order_date_x"] != df["order_date_y"]]
df[df["order_date_x"] == df["order_date_y"]].count()



order_id              3002
order_date_x          3002
customer_id           3002
origin_channel        3002
category              3002
sub-category          3002
product_id            3002
quantity              3002
order_date_y          3002
on_truck_scan_date    3002
processing_mode       3002
dtype: int64

In [14]:
df.drop("order_date_y", axis=1, inplace =True)


In [15]:
df.rename(columns = {"order_date_x": "order_date"}, inplace=True)
display(df)

,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,quantity,on_truck_scan_date,processing_mode
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-09,Standard Processing
1,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-09,Standard Processing
2,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-07,Express
3,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10002217,2.0,2019-01-09,Standard Processing
4,CA-2019-158211,2019-01-04,BP-11185,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10002026,6.0,2019-01-09,Standard Processing
...,...,...,...,...,...,...,...,...,...,...
2997,US-2020-158526,2020-12-29,KH-16360,Email,Power Muesli,Super Fibre Boost,OFF-BI-10002414,1.0,2021-01-04,Standard Processing
2998,CA-2020-115427,2020-12-30,EB-13975,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10002103,2.0,2021-01-06,Standard Processing
2999,CA-2020-126221,2020-12-30,CC-12430,Facebook,Power Muesli,Mega Protein,OFF-AP-10002457,2.0,2021-01-06,Standard Processing
3000,CA-2020-143259,2020-12-30,PO-18865,Email,Special Projects Muesil,Only Oats,TEC-PH-10004774,7.0,2021-01-06,Standard Processing


In [16]:
df["pretransportation_duration"] = df["on_truck_scan_date"] - df["order_date"]
df["pretransportation_duration"] = df["pretransportation_duration"].dt.days


In [17]:
df.describe()

,quantity,pretransportation_duration
count,3002.000000,3002.000000
mean,3.824450,6.105596
std,2.231853,2.475843
min,1.000000,0.000000
25%,2.000000,5.000000
50%,3.000000,7.000000
75%,5.000000,8.000000
max,14.000000,12.000000


In [18]:
df['weekday'] = df['order_date'].dt.weekday

In [19]:
df.head()

,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,quantity,on_truck_scan_date,processing_mode,pretransportation_duration,weekday
0,CA-2019-160304,2019-01-02,BM-11575,Sales,Special Projects Muesil,Only Oats,TEC-PH-10000455,2.0,2019-01-09,Standard Processing,7,2
1,CA-2019-105207,2019-01-03,BO-11350,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10004364,2.0,2019-01-09,Standard Processing,6,3
2,CA-2019-125206,2019-01-03,LR-16915,Email,Power Muesli,Steel Cut,OFF-ST-10003692,2.0,2019-01-07,Express,4,3
3,US-2019-116365,2019-01-03,CA-12310,Sales,Special Projects Muesil,Gluten Free,TEC-AC-10002217,2.0,2019-01-09,Standard Processing,6,3
4,CA-2019-158211,2019-01-04,BP-11185,Sales,Power Muesli,Super Fibre Boost,OFF-BI-10002026,6.0,2019-01-09,Standard Processing,5,4


In [20]:
# Create two labels of delivery date depending on express/ non/express processing(see flowchart)
same_arrival_day = [0, 2, 5, 6]
different_arrival_day = [1, 3, 4]

# Devide the dataframe into two groups with regard to these labels
same_day = df[df['weekday'].isin(same_arrival_day)]
different_days = df[df['weekday'].isin(different_arrival_day)]

# Doublecheck if the division is done correctly
check_difference = df['order_id'].count() - same_day['order_id'].count() - different_days['order_id'].count()
display(f'The Difference should be zero: ', check_difference)

'The Difference should be zero: '

0

In [21]:
# Compare descriptive statistics of each dataset
display('Same Day' ,same_day.describe())
display('Different Day' ,different_days.describe())

# Comparison:
# quantities: shows very equal distribution, though same_day orders are from 4 days and different days are from 3
 # Tuesday, Thursday and Friday are days with a lot of incoming orders


'Same Day'

,quantity,pretransportation_duration,weekday
count,1513.000000,1513.00000,1513.000000
mean,3.812293,6.42234,2.455387
std,2.183193,2.58955,2.319294
min,1.000000,0.00000,0.000000
25%,2.000000,5.00000,0.000000
50%,3.000000,7.00000,2.000000
75%,5.000000,9.00000,5.000000
max,14.000000,12.00000,6.000000


'Different Day'

,quantity,pretransportation_duration,weekday
count,1489.000000,1489.000000,1489.000000
mean,3.836803,5.783747,2.588986
std,2.280902,2.311491,1.228158
min,1.000000,0.000000,1.000000
25%,2.000000,5.000000,1.000000
50%,3.000000,6.000000,3.000000
75%,5.000000,8.000000,4.000000
max,14.000000,11.000000,4.000000


In [22]:
# We should analyse and visualize the days with the most incoming orders
# e.g. Barchart of orderquantities and weekdays


# Combine pretransportation_duration with standard_processing-column

In [26]:
# Investigate Delivery Time Assumption (average 3 days)
# Create merged df and campaign DF
campaign_merged = pd.merge(df, df_campaign , how='inner', on='order_id')

#Split into truck leaving the same day as the order and truck leaving on a different day
# matched_different_days = campaign_merged[campaign_merged['weekday'].isin(different_arrival_day)].reset_index().drop("index", axis=1)
# matched_same_day = campaign_merged[campaign_merged['weekday'].isin(same_arrival_day)].reset_index().drop("index", axis=1)

#Create new column for delivery duration for both DFs
# matched_different_days["delivery_duration"] = matched_different_days["arrival_scan_date"] - matched_different_days["on_truck_scan_date"]
# matched_same_day["delivery_duration"] = matched_same_day["arrival_scan_date"] - matched_same_day["on_truck_scan_date"]

campaign_merged.info()

#Simple Analysis on the two DFs
display(matched_different_days.describe())
display(matched_same_day.describe())

#Groupby weekday of truck scan

# Create new column for on truck scan weekday
campaign_merged['weekday_scan'] = campaign_merged['on_truck_scan_date'].dt.weekday
# Create new column for delivery duration
campaign_merged['delivery_duration'] = campaign_merged["arrival_scan_date"] - campaign_merged["on_truck_scan_date"]
campaign_merged['delivery_duration'].dt.days
# Create new column for total duration
campaign_merged['total_duration'] = campaign_merged["arrival_scan_date"] - campaign_merged["order_date"]
campaign_merged['delivery_duration'].dt.days

# Drop irrelevant columns
campaign_merged.drop(['quantity', 'weekday'], axis=1, inplace=True)
display(campaign_merged.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333 entries, 0 to 332
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    333 non-null    object        
 1   order_date                  333 non-null    datetime64[ns]
 2   customer_id                 333 non-null    object        
 3   origin_channel              333 non-null    object        
 4   category                    333 non-null    object        
 5   sub-category                333 non-null    object        
 6   product_id                  333 non-null    object        
 7   quantity                    333 non-null    float64       
 8   on_truck_scan_date          333 non-null    datetime64[ns]
 9   processing_mode             333 non-null    object        
 10  pretransportation_duration  333 non-null    int64         
 11  weekday                     333 non-null    int64         

,quantity,pretransportation_duration,weekday,delivery_duration
count,159.000000,159.000000,159.000000,159
mean,3.786164,5.855346,2.490566,4 days 16:18:06.792452830
std,2.309487,2.238434,1.211067,1 days 03:10:47.425865514
min,1.000000,0.000000,1.000000,1 days 00:00:00
25%,2.000000,5.000000,1.000000,4 days 00:00:00
50%,3.000000,6.000000,3.000000,5 days 00:00:00
75%,5.000000,8.000000,3.000000,5 days 00:00:00
max,13.000000,10.000000,4.000000,7 days 00:00:00


,quantity,pretransportation_duration,weekday,delivery_duration
count,174.000000,174.000000,174.000000,174
mean,3.913793,6.574713,3.011494,4 days 12:49:39.310344827
std,2.389412,2.773250,2.571494,1 days 06:09:53.799137346
min,1.000000,0.000000,0.000000,1 days 00:00:00
25%,2.000000,5.000000,0.000000,4 days 00:00:00
50%,3.000000,7.000000,2.000000,5 days 00:00:00
75%,5.000000,9.000000,6.000000,5 days 00:00:00
max,13.000000,12.000000,6.000000,7 days 00:00:00


,pretransportation_duration,weekday_scan,delivery_duration,total_duration
count,333.000000,333.000000,333,333
mean,6.231231,2.162162,4 days 14:29:11.351351351,10 days 20:02:09.729729729
std,2.553757,1.223814,1 days 04:47:16.148329809,2 days 20:41:39.045324279
min,0.000000,0.000000,1 days 00:00:00,3 days 00:00:00
25%,5.000000,2.000000,4 days 00:00:00,9 days 00:00:00
50%,7.000000,2.000000,5 days 00:00:00,11 days 00:00:00
75%,8.000000,2.000000,5 days 00:00:00,13 days 00:00:00
max,12.000000,4.000000,7 days 00:00:00,17 days 00:00:00


In [24]:
# Groupy on truck weekday
display(campaign_merged)
groupby = campaign_merged.groupby(by="weekday_scan")
groupby.mean()


,order_id,order_date,customer_id,origin_channel,category,sub-category,product_id,on_truck_scan_date,processing_mode,pretransportation_duration,arrival_scan_date,customer_name,weekday_scan,delivery_duration,total_duration
0,CA-2019-109666,2019-04-19,KM-16720,Email,Power Muesli,Hunger Buster,OFF-SU-10002522,2019-04-29,Standard Processing,10,2019-05-03,Kunst Miller,0,4 days,14 days
1,CA-2019-101329,2019-04-22,MM-17920,Email,Power Muesli,Super Mega Protein,OFF-AR-10000614,2019-05-01,Standard Processing,9,2019-05-08,Michael Moore,2,7 days,16 days
2,CA-2019-109057,2019-04-22,TT-21460,Facebook,Power Muesli,Steel Cut,OFF-ST-10002406,2019-05-01,Standard Processing,9,2019-05-07,Tonja Turnell,2,6 days,15 days
3,CA-2019-109869,2019-04-22,TN-21040,Facebook,Power Muesli,Hunger Buster,OFF-SU-10003505,2019-05-01,Standard Processing,9,2019-05-07,Tanja Norvell,2,6 days,15 days
4,CA-2019-113061,2019-04-22,EL-13735,Facebook,Toasted Muesli,With Fruit and Nuts,FUR-FU-10003975,2019-04-29,Standard Processing,7,2019-05-06,Ed Ludwig,0,7 days,14 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,CA-2020-125381,2020-04-25,SG-20605,Email,Special Projects Muesil,Gluten Free,TEC-AC-10000158,2020-05-01,Standard Processing,6,2020-05-08,Speros Goranitis,4,7 days,13 days
329,CA-2020-129707,2020-04-25,LH-16750,Email,Power Muesli,Super Mega Protein,OFF-AR-10000246,2020-05-04,Standard Processing,9,2020-05-08,Larry Hughes,0,4 days,13 days
330,CA-2020-141733,2020-05-07,RW-19540,Sales,Power Muesli,Mega Protein,OFF-AP-10001563,2020-05-13,Standard Processing,6,2020-05-15,Rick Wilson,2,2 days,8 days
331,US-2020-104451,2020-05-08,MM-18055,Email,Power Muesli,Mega Protein,OFF-AP-10000692,2020-05-13,Standard Processing,5,2020-05-15,Michelle Moray,2,2 days,7 days


,pretransportation_duration
weekday_scan,
0,5.913043
1,6.400000
2,6.628141
4,5.384615


In [25]:
# KPI:
# Company Level
# Average delivery duration (standard): now --> goal
# Average delivery duration (express): now --> goal
# --> stacked barchart from order to warehouse to delivery(process issues not relevant)

# Processing Level
# Average processing duration (standard): now --> goal
# Average processing duration (express): now --> goal 
# --> visualization (process issues not relevant, besides delivery)

# Warehouse Level
# Average warehouse duration (standard): now --> goal
# Average warehouse duration (express): now --> goal 
# --> visualization (all process issues are relevant) 